In [38]:
import pandas as pd

# ✅ 병합된 CSV 파일 경로
file_path = "../전처리_과정_데이터/1차전처리완료.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

print(df.head())

         발전구분  호기  지점명        위도         경도  설비용량(MW)  year  month  day  hour  \
0  (군산)삼랑진태양광   1  양산시  35.30737  129.02009       2.0  2022     12   31    20   
1  (군산)삼랑진태양광   1  양산시  35.30737  129.02009       2.0  2022     12   31    21   
2  (군산)삼랑진태양광   1  양산시  35.30737  129.02009       2.0  2022     12   31    22   
3  (군산)삼랑진태양광   1  양산시  35.30737  129.02009       2.0  2022     12   31    23   
4  (군산)삼랑진태양광   1  양산시  35.30737  129.02009       2.0  2023      1    1     0   

   ...  습도 QC플래그  적설(cm)  전운량(10분위)  풍속(m/s)  풍속 QC플래그  발전량(kWh)  발전효율  \
0  ...       NaN     NaN        0.0      1.2       NaN       0.0   0.0   
1  ...       NaN     NaN        1.0      1.4       NaN       0.0   0.0   
2  ...       NaN     NaN        2.0      2.7       NaN       0.0   0.0   
3  ...       NaN     NaN        0.0      1.3       NaN       0.0   0.0   
4  ...       NaN     NaN        0.0      0.4       NaN       0.0   0.0   

        태양고도         방위각  하늘상태  
0 -31.560920  262.601234   1.0  
1 

### 파생변수 추가

In [39]:
import numpy as np

# ✅ 이슬점 계산 함수 (0% 상대습도도 안전하게 처리)
def calculate_dew_point(temp_c, rh_percent):
    b = 17.62
    c = 243.12
    rh_percent = np.maximum(rh_percent, 0.1)  # 0%를 0.1%로 대체하여 log 방지
    gamma = (b * temp_c) / (c + temp_c) + np.log(rh_percent / 100.0)
    dew_point = (c * gamma) / (b - gamma)
    return dew_point

# ✅ 이슬점 컬럼 추가 (NaN 대체 대신 함수 내부에서 보정)
df['이슬점'] = calculate_dew_point(df['기온(°C)'], df['습도(%)'])

# ✅ 확인
print(df[['기온(°C)', '습도(%)', '이슬점']].head())



   기온(°C)  습도(%)       이슬점
0     2.5   53.0 -6.127064
1     1.2   57.0 -6.389528
2     2.8   52.0 -6.097851
3     1.5   55.0 -6.575349
4     0.6   59.0 -6.503007


In [40]:
# ✅ T - Td 컬럼 생성 (기온 - 이슬점)
df['T-Td'] = df['기온(°C)'] - df['이슬점']

In [41]:
import calendar

# ✅ 날짜 정보가 year, month, day로 되어 있다고 가정
df['date'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')

# ✅ 각 달의 총 일수 계산 (윤년 포함 정확하게 계산)
df['days_in_month'] = df['date'].dt.days_in_month

# ✅ 월.일 형식의 소수점 표현 (1일 → .0, 말일 → 거의 .999)
df['month_day'] = df['month'] + (df['day'] - 1) / df['days_in_month']

# ✅ 필요시 사인/코사인 인코딩도 추가
df['month_day_sin'] = np.sin(2 * np.pi * df['month_day'] / 12)
df['month_day_cos'] = np.cos(2 * np.pi * df['month_day'] / 12)

In [42]:
# ✅ hour 기반 sin/cos
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# ✅ month 기반 sin/cos
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

### 결측치 처리

In [43]:
# ✅ 제거 전 행 개수
before_count = len(df)

# ✅ 해당 컬럼 중 하나라도 결측이면 제거
df = df.dropna(subset=['하늘상태', '풍속(m/s)', '습도(%)', '기온(°C)'])

# ✅ 제거 후 행 개수
after_count = len(df)

# ✅ 제거된 행 수 출력
print(f"제거된 행 수: {before_count - after_count:,}개")
print(f"남은 행 수: {after_count:,}개")


제거된 행 수: 583,783개
남은 행 수: 2,003,417개


In [44]:
# ✅ 우선순위 컬럼 리스트
sorted_columns = [
    '발전구분', '호기', '지점명', '지점', '설비용량(MW)', '연식(년)', '고도', '위도', '경도',
    'year', 'month', 'month_sin', 'month_cos', 'day', 'month_day', 
    'month_day_sin', 'month_day_cos', 'hour', 'hour_sin', 'hour_cos',
    '태양고도', '방위각', '일사(MJ/m2)',
    '기온(°C)', '습도(%)', '이슬점', 'T-Td',
    '강수량(mm)', '하늘상태', '풍속(m/s)',
    '발전량(kWh)', '발전효율'
]

# ✅ 현재 df 컬럼 기준으로 정렬 가능한 컬럼만 추출
existing_columns = [col for col in sorted_columns if col in df.columns]

# ✅ 컬럼 순서 적용
df = df[existing_columns]


In [45]:
df.columns

Index(['발전구분', '호기', '지점명', '설비용량(MW)', '위도', '경도', 'year', 'month',
       'month_sin', 'month_cos', 'day', 'month_day', 'month_day_sin',
       'month_day_cos', 'hour', 'hour_sin', 'hour_cos', '태양고도', '방위각',
       '일사(MJ/m2)', '기온(°C)', '습도(%)', '이슬점', 'T-Td', '강수량(mm)', '하늘상태',
       '풍속(m/s)', '발전량(kWh)', '발전효율'],
      dtype='object')

In [46]:
save_path = "../전처리_과정_데이터/2차전처리완료.csv"
df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"✅ 저장 완료: {save_path}")


✅ 저장 완료: ../전처리_과정_데이터/2차전처리완료.csv
